In [2]:
import pandas as pd
import sys
import os
import re
sys.path.append(os.path.abspath('../scripts'))  #esto lo colocamos para que el python nos lea la ruta bien
from utils import process_null_values, tokenice_and_lemati # Importa la funcinn desde utils
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
dataset = pd.read_csv('../data_process/FinalDataTrainer.csv')

In [4]:
dataset.describe()

,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [14]:
dataset.head()
dataset.dtypes

rating             int64
processed_text    object
sentiment         object
dtype: object

In [16]:
conteo_reseñas = dataset.groupby('rating').size()
conteo_reseñas

rating
1    7923
2    1715
3    1633
4    1591
5    1097
dtype: int64

In [ ]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay(no hay de otra)
#eliminar muchos registros para balancear la data
condition = dataset['rating'] == 5
num_to_remove = 50000
rows_to_remove = dataset[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
dataset = dataset.drop(rows_to_remove.index)

dataset.describe()


,rating,target
count,50000.000000,43225.000000
mean,2.950620,0.536379
std,1.542838,0.498681
min,1.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,1.000000
75%,4.000000,1.000000
max,5.000000,1.000000


In [ ]:
analyzer = SentimentIntensityAnalyzer()

def analizar_vader(texto):
    # Obtener el puntaje de polaridad
    sentiment_score = analyzer.polarity_scores(texto)
    compound_score = sentiment_score['compound']

    # Evaluar el sentimiento en base al puntaje "compound"
    if compound_score >= 0.5:
        return 'contento'  # Muy positivo
    elif compound_score >= 0.05:
        return 'positivo'  # Positivo
    elif compound_score <= -0.8:
        return 'enojado'  # Muy negativo
    elif compound_score <= -0.5:
        return 'insatisfecho'  #"enojado"
    elif compound_score <= -0.05:
        return 'negativo'  # Negativo
    else:
        return 'neutral'  # Neutral

In [52]:
#formateando las columnas
dataset = dataset.rename(columns={'content': 'processed_text'})
dataset.drop(['reviewId'], axis=1, inplace=True) #eliminando una columna inesesaria 
dataset.drop(['sentiment'], axis=1, inplace=True) #eliminando una columna inesesaria sentiment

In [27]:
dataset = dataset.rename(columns={'rating': 'score'})

In [28]:
dataset['processed_text'] = dataset['processed_text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
dataset

,score,processed_text,sentiment
0,1,bad talented illustrations context degrade sit...,negativo
1,2,tape measure level work good extend full 25 fe...,negativo
2,1,place large order seller include plant ship fa...,insatisfecho
3,1,no idea thing work nt help no specifications p...,neutral
4,1,product arrive canister full someone else s di...,negativo
...,...,...,...
13954,2,two one get use rare occasion around year half...,negativo
13955,5,ve cars little year son play daily use abuse m...,negativo
13956,1,could not play product dvd player must hd dvd ...,negativo
13957,5,not qualify review great classic love read aut...,positivo


In [5]:
# Verificar si hay valores nulos (NaN) en la columna
process_null_values(dataset)


Total de registros:
la columna score tiene un Total de: 13473 valores vacios
--------------------------------------------------------------------------------
Porcentaje de registros:
la columna score tiene un Porcentaje de: 18.594989993789248 valores vacios
--------------------------------------------------------------------------------
Si las columnas son irrelevantes, podemos hacer algo con sus datos, que te gustaria eliminarlos o los conservamos?


data limpiada maestro, que tengas un buen dia :)


### generar sentimientos

In [19]:
dataset['sentiment'] = dataset['processed_text'].apply(analizar_vader)

### Funcion para lemtizar

In [20]:
#plematizar
dataset['processed_text'] = dataset['review'].apply(tokenice_and_lemati)


KeyError: 'review'

In [20]:
dataset.drop(['review'], axis=1, inplace=True) #eliminando una columna inesesaria sentiment

In [6]:
dataset.head(30)

,processed_text,sentiment,score
0,bad talented illustrations context degrade sit...,negativo,NaN
1,tape measure level work good extend full 25 fe...,negativo,NaN
2,place large order seller include plant ship fa...,insatisfecho,NaN
3,no idea thing work n't help no specifications ...,neutral,NaN
4,product arrive canister full someone else 's d...,negativo,NaN
5,concept teach book methods use evaluate compan...,insatisfecho,NaN
6,book weak every aspect character shallow prose...,insatisfecho,NaN
7,never ever receive seller n't acknowledge requ...,negativo,NaN
8,dvd 's fan wait release 8 movies extend versio...,negativo,NaN
9,product work describe problem arise motion adj...,negativo,NaN


In [29]:
conteo_sentimientos = dataset.groupby('sentiment').size()
conteo_sentimientos

sentiment
enojado         2312
insatisfecho    3807
negativo        5326
neutral         2028
positivo         486
dtype: int64

In [ ]:
#limpiado datos positivos
condition = dataset['sentiment'] == 'contento'
num_to_remove =  3000
rows_to_remove = dataset[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
dataset = dataset.drop(rows_to_remove.index)

dataset.describe()

,score
count,78496.000000
mean,3.458036
std,1.501689
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,5.000000


In [42]:
dataset.to_csv('../data_process/FinalDataTrainer.csv', index=False)

In [5]:
#unir los 3 dataset
dataset1 = pd.read_csv('../data_process/NetflixFinal.csv')
dataset2 = pd.read_csv('../data_process/CellFinals.csv')
dataset3 = pd.read_csv('../data_process/CandyFinal.csv')

df_unido = pd.concat([dataset1, dataset2, dataset3], axis=0, ignore_index=True)

In [30]:
#ultima union de datasets
dataset1 = pd.read_csv('../data_process/NegativesView.csv')
dataset2 = pd.read_csv('../data_process/FinalDataTrainer.csv')

df_unido = pd.concat([dataset1, dataset2], axis=0, ignore_index=True)

In [34]:
df_unido.drop(['rating'], axis=1, inplace=True) #eliminando una columna inesesaria 

In [47]:
df_unido.to_csv('../data_process/FinalDataTrainer.csv', index=False)

In [46]:
df_unido.describe()


,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [44]:
conteo_sentimientos = df_unido.groupby('sentiment').size()
conteo_sentimientos

sentiment
contento        13101
enojado          4204
insatisfecho    10316
negativo        18183
neutral          9015
positivo        17636
dtype: int64

In [43]:
#limpiado datos positivos
condition = df_unido['sentiment'] == 'positivo'
num_to_remove =  4000
rows_to_remove = df_unido[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
df_unido = df_unido.drop(rows_to_remove.index)

df_unido.describe()

,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [50]:
df_unido.describe()

,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [51]:
df_unido.to_csv('../data_process/FinalDataTrainer.csv', index=False)
